In [1]:
from os.path import join

from openscm.units import unit_registry
from openscm.highlevel import OpenSCMDataFrame, run

<IPython.core.display.Javascript object>

In [2]:
import matplotlib
%matplotlib inline

In [3]:
DATA_PATH = join("..", "data", "rcps")

In [4]:
rcp26 = OpenSCMDataFrame(join(DATA_PATH, "rcp26_emissions.csv"))
rcp26.head()

INFO:root:Reading `../data/rcps/rcp26_emissions.csv`


,model,scenario,region,variable,unit,todo,climate_model,time,value
0,IMAGE,RCP26,World,Emissions|BC,Mt BC / yr,SET,unspecified,1765-07-12 00:00:00,0.000000
39,IMAGE,RCP26,World,Emissions|BC,Mt BC / yr,SET,unspecified,1766-07-12 00:00:00,0.106998
78,IMAGE,RCP26,World,Emissions|BC,Mt BC / yr,SET,unspecified,1767-07-12 00:00:00,0.133383
117,IMAGE,RCP26,World,Emissions|BC,Mt BC / yr,SET,unspecified,1768-07-12 00:00:00,0.159847
156,IMAGE,RCP26,World,Emissions|BC,Mt BC / yr,SET,unspecified,1769-07-12 00:00:00,0.186393


In [ ]:
rcps = OpenSCMDataFrame(rcp26.data.copy())  # TODO: add copy method
for rcp in ["rcp45", "rcp60", "rcp85"]:
    rcps.append(
        join(DATA_PATH, "{}_emissions.csv".format(rcp)),
        inplace=True
    )

INFO:root:Reading `../data/rcps/rcp45_emissions.csv`
INFO:root:Reading `../data/rcps/rcp60_emissions.csv`
INFO:root:Reading `../data/rcps/rcp85_emissions.csv`


In [ ]:
rcps.scenarios()

0    RCP26
1    RCP45
2    RCP60
3    RCP85
Name: scenario, dtype: object

In [ ]:
res = run(
    rcps, 
    {
        "PH99": [
            {"beta": ("World", 0.8 * unit_registry("ppm/GtC"))},
            {"beta": ("World", 0.3 * unit_registry("ppm/GtC"))},
        ]
    },
);

running PH99

running RCP26


| |                                #                  | 1 Elapsed Time: 0:00:06


running RCP45


| |               #                                   | 1 Elapsed Time: 0:00:08


running RCP60


/ |#                                                  | 0 Elapsed Time: 0:00:00

In [ ]:
res.filter(
    variable="Surface Temperature",
    region="World",
).line_plot(x="time", color="scenario");

In [ ]:
res.meta